In [ ]:
from sys import path

from numpy import where
from pandas import DataFrame

path.append("../../../")
from componentes.pipeline.pipeline import Pipeline
from componentes.artefato.arquivo import Arquivo
from componentes.container.container_bronze import ContainerBronze
from componentes.container.container_silver import ContainerSilver
from componentes.container.categoria_de_armazenamento import \
    CategoriaDeArmazenamento
from utils.transformacoes_dataframe import remover_espacamento_das_colunas

In [ ]:
class CNAEBronzeParaSilver(Pipeline):
    """
    Pipeline de dados responsável por executar o processamento de artefatos
    do CNAE de categoria bronze para silver
    """
    def _transformar_dados(self, dataframe: DataFrame) -> DataFrame:
        """
        Realiza a normalização e tratamento de dados do artefato
        de referência para a tabela de CNAE
        """
        self._logger.log_mensagem(mensagem="Iniciando transformações")
        dataframe = self._criar_coluna_codigo_cnae(dataframe=dataframe)
        dataframe = self._criar_coluna_tipo_cnae(dataframe=dataframe)
        dataframe = self._remover_colunas_desnecessarias(dataframe=dataframe)
        dataframe = remover_espacamento_das_colunas(dataframe=dataframe)
        return dataframe
    
    def _criar_coluna_codigo_cnae(self, dataframe: DataFrame) -> DataFrame:
        """
        Cria a coluna codigo_cnae unificando as colunas já existentes
        divisao, grupo e classe
        """
        self._logger.log_mensagem(mensagem="Criando coluna codigo_cnae")
        dataframe["cogido_cnae"] = (
            dataframe["divisao"]
            .fillna(value=dataframe["grupo"])
            .fillna(value=dataframe["classe"]))
        return dataframe

    def _criar_coluna_tipo_cnae(self, dataframe: DataFrame) -> DataFrame:
        """
        Cria a coluna tipo_cnae se baseando no preenchimento das colunas já
        divisao, grupo e classe
        """
        self._logger.log_mensagem(mensagem="Criando coluna tipo_cnae")
        dataframe["tipo_cnae"] = (
            where(dataframe["divisao"].isnull() == False, "Divisão",
            where(dataframe["grupo"].isnull() == False, "Grupo",
            where(dataframe["classe"].isnull() == False, "Classe", None))))
        return dataframe

    def _remover_colunas_desnecessarias(self, dataframe: DataFrame) -> DataFrame:
        """
        Remoção de colunas indesejadas
        """
        self._logger.log_mensagem(mensagem="Removendo colunas desnecessárias")
        return dataframe.drop(columns=["divisao", "grupo", "classe"])

In [ ]:
artefato_de_entrada = Arquivo(
    nome="construcao",
    categoria_de_armazenamento=CategoriaDeArmazenamento.BRONZE,
    fonte="cnae")
container_de_entrada = ContainerBronze()
artefato_de_saida = Arquivo(
    nome="construcao",
    categoria_de_armazenamento=CategoriaDeArmazenamento.SILVER,
    fonte="cnae")
container_de_saida = ContainerSilver()
pipeline = CNAEBronzeParaSilver(
    artefato_de_entrada=artefato_de_entrada,
    container_de_entrada=container_de_entrada,
    artefato_de_saida=artefato_de_saida,
    container_de_saida=container_de_saida)
pipeline.executar_etl()